### gpu 활용 가능한지 확인

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# GPU 이름 체크(cuda:0에 연결된 그래픽 카드 기준)
print(torch.cuda.get_device_name(device = 0)) # 'NVIDIA TITAN X (Pascal)'

# 사용 가능 GPU 개수 체크
print(torch.cuda.device_count()) # 3

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

### 필요한 파이썬 패키지 설치

In [ ]:
!pip install transformers datasets numpy accelerate pandas tokenizers matplotlib

### 워드 토큰화 및 임베딩 확인

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base", force_download=True)

sentence = "안녕하세요. 오늘은 LLM 파인 튜닝을 배워볼게요."

In [ ]:
tokens = tokenizer.tokenize(sentence)

In [ ]:
tokens

In [ ]:
# 인덱스로 변환 (토큰 ID)
token_ids = tokenizer.encode(sentence)
print("Token IDs:", token_ids)

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
from transformers import AutoTokenizer

# 한국어 모델 로드 (KoBERT)
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

# 한국어 문장 토큰화
sentence = "안녕하세요, 한국어 LLM 실습입니다."
tokens = tokenizer.tokenize(sentence)
print("Tokens:", tokens)

# 토큰 인덱스로 변환
token_ids = tokenizer.encode(sentence)
print("Token IDs:", token_ids)


In [ ]:
!pip install matplotlib

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset
import torch
from transformers import TrainerCallback


# Custom Callback to log every step
class CustomCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Step: {state.global_step}, Training Loss: {logs.get('loss', 'N/A')}")

# 한국어 데이터셋 로드 (NSMC 영화 리뷰 데이터셋)
dataset = load_dataset('nsmc')

# 사전 학습된 한국어 BERT 모델 로드 및 GPU로 이동
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

# 토큰화 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['document'], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",       # 매 스텝마다 평가
    save_strategy="steps",             # 매 스텝마다 모델 저장
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',              # 로그 디렉토리
    logging_steps=10,                  # 로그 출력 간격
    log_level="info",                  # 로그 레벨 설정
    save_total_limit=2,                # 모델 저장 횟수 제한
    save_steps=500,                    # 500 스텝마다 모델 저장
    eval_steps=500,                    # 500 스텝마다 평가
    load_best_model_at_end=True,       # 학습 종료 시 베스트 모델 로드
    fp16=True,                         # Mixed precision 학습
    report_to="none",                  # 콘솔에 로그 출력
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    callbacks=[CustomCallback()],  # 콜백 추가
)

# 학습 시작
trainer.train()

# 모델 저장
model.save_pretrained("./bert_korean_model")
tokenizer.save_pretrained("./bert_korean_model")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPT 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2").to(device)
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

# 한국어 데이터셋 로드 (NSMC 예시)
dataset = load_dataset("nsmc")

# 토큰화 및 패딩 설정
def tokenize_function(examples):
    return tokenizer(examples['document'], padding="max_length", truncation=True, max_length=128)  # 고정된 시퀀스 길이 설정

# 토큰화된 데이터셋 생성
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 모델 입력에 맞는 데이터셋 형식으로 변환
def group_texts(examples):
    # GPT 모델은 시퀀스가 연결된 상태로 학습되므로 여러 텍스트를 하나의 시퀀스로 그룹화
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= 128:  # 여기서는 max_length와 일치해야 함
        total_length = (total_length // 128) * 128
    result = {k: [t[i: i + 128] for i in range(0, total_length, 128)] for k, t in concatenated_examples.items()}
    return result

# 데이터셋을 GPT 모델 입력에 맞게 변환
tokenized_datasets = tokenized_datasets.map(group_texts, batched=True)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./gpt_korean_results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./gpt_logs',
    logging_steps=50,
    save_total_limit=2,
    fp16=True,  # Mixed precision 학습 (FP16) GPU 속도 향상
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# 파인튜닝 시작 (GPU에서 실행)
trainer.train()

# 모델 저장
model.save_pretrained("./gpt_korean_model")
tokenizer.save_pretrained("./gpt_korean_model")


# 다음 단어 예측 실습
input_sentence = "오늘 날씨는"
inputs = tokenizer(input_sentence, return_tensors="pt")
outputs = model.generate(**inputs, max_length=50, do_sample=True, top_k=50)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("생성된 문장:", generated_text)


bert pretrained model 예측

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 사전 학습된 한국어 BERT 모델 로드 (분류용)
model_name = "beomi/kcbert-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPU 설정 (필요 시 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 예측할 문장
input_sentence = "행복"
inputs = tokenizer(input_sentence, return_tensors="pt").to(device)

# 모델 예측
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    print(logits)
    predicted_class = torch.argmax(logits, dim=1).item()

# 예측 결과 출력
print(f"예측 결과: {'긍정' if predicted_class == 1 else '부정'}")


gpt pretrained model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 사전 학습된 GPT 모델 로드 (텍스트 생성용)
model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPU 설정 (필요 시 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 문장 입력
input_sentence = "오늘 날씨는"
inputs = tokenizer(input_sentence, return_tensors="pt").to(device)

# 텍스트 생성
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=100, do_sample=True, top_k=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 생성된 텍스트 출력
print("생성된 텍스트:", generated_text)
